In [2]:
import napari
import tkinter as tk
from tkinter import filedialog
from scipy.ndimage import median_filter
from skimage.io import imread  # Import the function to read TIFF files
import skimage
from skimage import img_as_ubyte
import os
import sys
import numpy as np
import pandas as pd
import cv2
import cc3d
#from ipynb.fs.full.utils_Main_Baseline import create_tiff, mkdir_p
from tifffile import imread
from skimage import io, measure, color, filters, morphology
from skimage.measure import regionprops
from skimage.segmentation import mark_boundaries
from skimage.metrics import peak_signal_noise_ratio, structural_similarity, mean_squared_error
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt, patches
from PIL import Image
from scipy import ndimage
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import median_filter
from tifffile import imsave
import os
import numpy as np
import matplotlib.pyplot as plt

# Create a function to open Napari viewer
def open_napari_viewer(tiff_data):
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(tiff_data, name='TIFF Data')
        
def load_tiff_data():
    try:
        file_path = filedialog.askopenfilename(filetypes=[("TIFF Files", "*.tif")])
        if file_path:
            tiff_data = imread(file_path)
            open_napari_viewer(tiff_data)
    except Exception as e:
        messagebox.showerror("Error", f"Failed to load image: {e}")
        
import os
import sys
import numpy as np
import pandas as pd
import cv2
import cc3d
#from ipynb.fs.full.utils_Main_Baseline import create_tiff, mkdir_p
from tifffile import imread
from skimage import io, measure, color, filters, morphology
from skimage.measure import regionprops
from skimage.segmentation import mark_boundaries
from skimage.metrics import peak_signal_noise_ratio, structural_similarity, mean_squared_error
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt, patches
from PIL import Image
from scipy import ndimage
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import median_filter
from tifffile import imsave
import os
import numpy as np
import matplotlib.pyplot as plt

#%run utils_Main_Baseline.ipynb

#Import functions defined in my Utils notebook
#from ipynb.fs.full.utils_Main_Baseline import create_tiff, mkdir_p
#import ipynb.fs.full.utils_Main_Baseline

#Import functions defined in my Utils notebook
#from ipynb.fs.full.utils_Main_Baseline import apply_threshold_save

import pandas as pd
import skimage.measure as measure

#Import general functions
import os
import numpy as np
from tifffile import imread
from skimage.filters import median
from numpy import mean
import matplotlib.pyplot as plt
from PIL import Image
from skimage.morphology import (erosion, dilation, closing, opening,
                                area_closing, area_opening, ball, remove_small_objects)
from skimage.morphology import white_tophat, black_tophat
import cc3d
import cv2
from scipy import ndimage
from skimage import io, measure
from sklearn.metrics import mean_squared_error
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity, mean_squared_error
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from skimage.measure import regionprops
import numpy as np

import sys

import sys

def hsv_to_rgb(h, s, v):  # TODO: remove colorsys dependency
    """
    Converts a color from HSV to RGB

    Parameters
    ----------
    h : float
    s : float
    v : float

    Returns
    -------
    numpy.ndarray
        The converted color in RGB space.
    """
    i = np.floor(h*6.0)
    f = h * 6 - i
    p = v * (1 - s)
    q = v * (1 - s * f)
    t = v * (1 - s * (1 - f))
    i = i % 6

    if i == 0:
        rgb = (v, t, p)
    elif i == 1:
        rgb = (q, v, p)
    elif i == 2:
        rgb = (p, v, t)
    elif i == 3:
        rgb = (p, q, v)
    elif i == 4:
        rgb = (t, p, v)
    else:
        rgb = (v, p, q)

    return np.array(rgb, dtype=np.float32)

def get_distinct_colors(n, min_sat=.5, min_val=.5):
    """
    Generates a list of distinct colors, evenly separated in HSV space.

    Parameters
    ----------
    n : int
        Number of colors to generate.
    min_sat : float
        Minimum saturation.
    min_val : float
        Minimum brightness.

    Returns
    -------
    numpy.ndarray
        Array of shape (n, 3) containing the generated colors.

    """
    huePartition = 1.0 / (n + 1)
    hues = np.arange(0, n) * huePartition
    saturations = np.random.rand(n) * (1-min_sat) + min_sat
    values = np.random.rand(n) * (1-min_val) + min_val
    return np.stack([hsv_to_rgb(h, s, v) for h, s, v in zip(hues, saturations, values)], axis=0)

def colorize_segmentation(seg, ignore_label=None, ignore_color=(0, 0, 0)):
    """
    Randomly colorize a segmentation with a set of distinct colors.

    Parameters
    ----------
    seg : numpy.ndarray
        Segmentation to be colorized. Can have any shape, but data type must be discrete.
    ignore_label : int
        Label of segment to be colored with ignore_color.
    ignore_color : tuple
        RGB color of segment labeled with ignore_label.

    Returns
    -------
    numpy.ndarray
        The randompy colored segmentation. The RGB channels are in the last axis.
    """
    assert isinstance(seg, np.ndarray)
    assert seg.dtype.kind in ('u', 'i')
    if ignore_label is not None:
        ignore_ind = seg == ignore_label
    seg = seg - np.min(seg)
    colors = get_distinct_colors(np.max(seg) + 1)
    np.random.shuffle(colors)
    result = colors[seg]
    if ignore_label is not None:
        result[ignore_ind] = ignore_color
    return result


def get_bounding_boxes_image(labelled):
    # Determine slice with the most labels
    counts = np.sum(labelled > 0, axis=(1, 2))  # count non-zero pixels in each slice
    max_slice_idx = np.argmax(counts)

    props = regionprops(labelled[max_slice_idx])

    fig, ax = plt.subplots()
    ax.imshow(labelled[max_slice_idx], cmap='gray')

    for prop in props:
        # Get 2D bounding box coordinates
        minr, minc, maxr, maxc = prop.bbox

        # Draw rectangle around the object
        rect = patches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                 fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)

    return fig


import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage import color
from skimage import measure
import cc3d  
from skimage.io import imsave


########################
def convert_hsv_to_rgb(h: float, s: float, v: float) -> np.ndarray:
    """
    Converts a color from HSV to RGB.

    Parameters:
    - h, s, v : float : The hue, saturation, and value components of the color.

    Returns:
    - numpy.ndarray : The converted color in RGB space.
    """
    color_division = np.floor(h * 6.0)
    fractional_part = h * 6 - color_division
    value_component = v * (1 - s)
    first_value = v * (1 - s * fractional_part)
    second_value = v * (1 - s * (1 - fractional_part))
    color_division = color_division % 6

    if color_division == 0:
        rgb_values = (v, second_value, value_component)
    elif color_division == 1:
        rgb_values = (first_value, v, value_component)
    elif color_division == 2:
        rgb_values = (value_component, v, second_value)
    elif color_division == 3:
        rgb_values = (value_component, first_value, v)
    elif color_division == 4:
        rgb_values = (second_value, value_component, v)
    else:
        rgb_values = (v, value_component, first_value)

    return np.array(rgb_values, dtype=np.float32)

#################################
def generate_unique_colors(num_colors: int, min_saturation: float = .5, min_brightness: float = .5) -> np.ndarray:
    """
    Generates a list of distinct colors, evenly separated in HSV space.

    Parameters:
    - num_colors : int : Number of colors to generate.
    - min_saturation, min_brightness : float : Minimum saturation and brightness.

    Returns:
    - numpy.ndarray : Array of shape (num_colors, 3) containing the generated colors.
    """
    hue_spacing = 1.0 / (num_colors + 1)
    hue_values = np.arange(0, num_colors) * hue_spacing
    saturation_values = np.random.rand(num_colors) * (1 - min_saturation) + min_saturation
    brightness_values = np.random.rand(num_colors) * (1 - min_brightness) + min_brightness
    
    return np.stack([convert_hsv_to_rgb(hue, saturation, brightness) for hue, saturation, brightness in zip(hue_values, saturation_values, brightness_values)], axis=0)

##############################
def apply_random_colors_to_segmentation(segmentation_data: np.ndarray, excluded_label: int = None, excluded_color: tuple = (0, 0, 0)) -> np.ndarray:
    """
    Applies random colors to a segmentation with distinct colors.

    Parameters:
    - segmentation_data : numpy.ndarray : The segmentation to be colorized. Must be discrete.
    - excluded_label : int : Label of segment to be colored with excluded_color.
    - excluded_color : tuple : RGB color of segment labeled with excluded_label.

    Returns:
    - numpy.ndarray : The segmentation with randomly applied colors and RGB channels in the last axis.
    """
    if excluded_label is not None:
        excluded_indices = segmentation_data == excluded_label
    segmentation_data = segmentation_data - np.min(segmentation_data)
    
    distinct_colors = generate_unique_colors(np.max(segmentation_data) + 1)
    np.random.shuffle(distinct_colors)
    
    colored_segmentation = distinct_colors[segmentation_data]
    
    if excluded_label is not None:
        colored_segmentation[excluded_indices] = excluded_color
        
    return colored_segmentation

###############################
def visualize_segments_with_bounding_boxes(labeled_data: np.ndarray) -> plt.Figure:
    """
    Visualizes segments with bounding boxes.

    Parameters:
    - labeled_data : numpy.ndarray : The labeled segmentation.

    Returns:
    - plt.Figure : A figure with segments highlighted by bounding boxes.
    """
    segment_counts_per_slice = np.sum(labeled_data > 0, axis=(1, 2))
    slice_with_most_segments = np.argmax(segment_counts_per_slice)

    regions_in_slice = regionprops(labeled_data[slice_with_most_segments])

    fig, ax = plt.subplots()
    ax.imshow(labeled_data[slice_with_most_segments], cmap='gray')

    for region in regions_in_slice:
        min_row, min_col, max_row, max_col = region.bbox
        bounding_box = patches.Rectangle((min_col, min_row), max_col - min_col, max_row - min_row, fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(bounding_box)

    return fig

#################################
def initialize_data(astrocyte3d):
    t_dim, y_dim, x_dim = astrocyte3d.shape
    
    default_config = {
            "time_start_index": 300,
            "time_end_index": 400,
            "y_start_index": 0,
            "y_end_index": y_dim,
            "x_start_index": 0,
            "x_end_index": x_dim,
            "median_filter_frames": 5,
            "median_filter_width": 3,
            "median_filter_height": 3,
            "jupyter_display_frame": 70,
            "background_substraction_neighbors": 10,
            "manual_threshold": 2500,
            "threshold_percentage": 95,
            "morph_op_element_size_xy": 2,
            "morph_op_element_size_t": 5,
            "connected_components_count": 26,
            "min_volume_pixel_size": 500
        }
    
    parameter_explanations = {
        "time_start_index": "Starting time index for cropping.",
        "time_end_index": "Ending time index for cropping.",
        "y_start_index": "Starting y index for cropping.",
        "y_end_index": "Ending y index for cropping.",
        "x_start_index": "Starting x index for cropping.",
        "x_end_index": "Ending x index for cropping.",
        "median_filter_frames": "How many frames for Median Filtering?",
        "median_filter_width": "Kernel width for Median Filtering",
        "median_filter_height": "Kernel height for Median Filtering",
        "jupyter_display_frame": "Frame to look at in jupyter",
        "background_substraction_neighbors": "How many +/- and neighbors to consider for Background Substraction",
        "manual_threshold": "Manual Setting of the Threshold",
        "threshold_percentage": "Threshold Percentage",
        "morph_op_element_size_xy": "Structure Element Size in x,y for Morphological Operations",
        "morph_op_element_size_t": "Structure Element Size in t for Morphological Operations",
        "connected_components_count": "How many Connected Components should be used?",
        "min_volume_pixel_size": "Minimum volume pixel size"
    }
    
    return default_config, parameter_explanations

#######################################
def create_widgets(default_config, parameter_explanations):
    use_default_checkbox = widgets.Checkbox(description="Use Default Parameters")
    input_widgets = {}
    
    for param, default_value in default_config.items():
        description = param + ":"
        input_widget = widgets.IntText(value=default_value, description=description)
        explanation_widget = widgets.HTML(value=parameter_explanations.get(param, ""), 
                                          layout=widgets.Layout(margin="0 0 0 15px"))
        
        hbox = widgets.HBox([input_widget, explanation_widget])
        input_widgets[param] = hbox
    
    return use_default_checkbox, input_widgets

#######################################
def display_widgets(use_default_checkbox, input_widgets):
    display(use_default_checkbox)
    for hbox in input_widgets.values():
        display(hbox)

    apply_button = widgets.Button(description="Apply Parameters")
    display(apply_button)
    
    return apply_button

######################################
def apply_parameters(b, input_widgets):
    config = {}
    for param, hbox in input_widgets.items():
        config[param] = hbox.children[0].value
    print("Config:", config)
    
######################################    
def load_dataset(dataset_name: str) -> np.ndarray:
    """
    Load a dataset given its name.

    Parameters:
    - dataset_name : str : Name identifier of the dataset.

    Returns:
    - numpy.ndarray : Loaded dataset.
    """
    base_path = "/export/scratch/mkunfeld/DataSets/3D/"
    datasets = {
        "ADRA2a219": "ADRA2a219-SSCx-roi1-Sham7d-Zm69-LoIso-Baseline-3_Registered.tif",
        "GMIPKO459": "GMIPKO459-SSCx-roi1-dps38-SNI7d-Zm45-lowiso-basal-4-rep80_Registered.tif",
        "Adra2a#392": "Adra2a#392-hlSS1-ROI4-PreSNI-2d-Zm2-LoIso-Bl9-Vf10-Ac11-Treg.tif",
        "GMIPKO455": "GMIPKO455-SSCx-roi1-dps29-preSNI-Zm4-4-Treg.tif",
        "GMIP355": "GMIP355-SSCx-roi1-dps51-21dSNI-Zm2-lowiso-1-2-3-Treg.tif",
        "Adra2a#219": "Adra2a#219-SSCx-roi1-Sham21d-Zm6.9-11-12-Treg_corr_5steps.tif",
    }

    if dataset_name not in datasets:
        raise ValueError(f"Unknown dataset name: {dataset_name}")

    dataset_path = os.path.join(base_path, datasets[dataset_name]) 
    data = imread(dataset_path)
    return data, dataset_name, dataset_path  # Return both the data and the chosen dataset name

########################################
def get_parameter_string(config, key_parameters):
    """
    Convert selected parameters from the config dictionary into a string representation.
    """
    return '_'.join([f"{param}={config[param]}" for param in key_parameters if param in config])

#########################################
def generate_unique_subfolder_path(main_folder_path, dataset_name, parameter_string):
    """
    Generate a unique subfolder path based on the dataset name and parameters.
    """
    subfolder_path = os.path.join(main_folder_path, f"{dataset_name}_{parameter_string}")
    
    counter = 1
    while os.path.exists(subfolder_path):
        subfolder_path = os.path.join(main_folder_path, f"{dataset_name}_{parameter_string}_{counter}")
        counter += 1

    return subfolder_path

#########################################
def create_folder_if_not_exists(folder_path):
    """
    Create a directory if it doesn't exist.
    """
    if not os.path.exists(folder_path):
        os.mkdir(folder_path)

##########################################        
def main_folder_for_today(base_path):
    """
    Get the main folder path for today's date.
    """
    today = datetime.now().strftime('%Y%m%d')
    return os.path.join(base_path, f"Results/{today}")

##########################################
def get_dataset_name_from_path(dataset_path):
    """
    Extract dataset name from the provided file path.
    """
    return os.path.splitext(os.path.basename(dataset_path))[0]

#########################################
# List of parameters to include in the subfolder name
key_parameters = [
    'time_start_index', 'time_end_index', 'y_start_index', 'y_end_index', 'x_start_index', 'x_end_index',
    'median_filter_frames', 'median_filter_width', 'median_filter_height', 'background_substraction_neighbors',
    'threshold_percentage', 'morph_op_element_size_xy', 'morph_op_element_size_t', 'connected_components_count', 'min_volume_pixel_size'
]

##########################################
def get_parameter_string(config, key_parameters):
    """
    Convert selected parameters from the config dictionary into a string representation.
    """
    # Shorter versions of the keys
    short_keys = {
        'time_start_index': 't1',
        'time_end_index': 't2',
        'y_start_index': 'y1',
        'y_end_index': 'y2',
        'x_start_index': 'x1',
        'x_end_index': 'x2',
        'median_filter_frames': 'bw_t',
        'median_filter_width': 'bw_y',
        'median_filter_height': 'bw_x',
        'background_substraction_neighbors': 'Nframes',
        'threshold_percentage': 't_perc',
        'morph_op_element_size_xy': 'SE_xy',
        'morph_op_element_size_t': 'SE_t',
        'connected_components_count': 'Con',
        'min_volume_pixel_size': 'min_area'
    }

    return '_'.join([f"{short_keys[param]}={config[param]}" for param in key_parameters if param in config])

###############################
def apply_median_filtering(data, kernel):
    return median_filter(data, footprint=kernel)

###############################
def calculate_background(data, num_frames):
    total_frames = data.shape[0]
    background = np.zeros(data.shape, 'uint16')

    for i in range(0, total_frames):
        if i - num_frames < 0:
            background[i, :, :] = np.min(data[0:(i + num_frames + 1), :, :], axis=0, keepdims=True)
        elif i + num_frames + 1 > total_frames:
            background[i, :, :] = np.min(data[(i - num_frames):, :, :], axis=0, keepdims=True)
        else:
            background[i, :, :] = np.min(data[(i - num_frames):(i + num_frames + 1), :, :], axis=0, keepdims=True)
    
    return background

################################
def create_and_save_subfolder(parent_folder, subfolder_name, file_name, data):
    subfolder_path = os.path.join(parent_folder, subfolder_name)
    os.makedirs(subfolder_path, exist_ok=True)
    imsave(os.path.join(subfolder_path, file_name + '.tif'), data)

################################    
def display_images_side_by_side(image1, image2, title1, title2, frame):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    axes[0].imshow(image1[frame], cmap="gray")
    axes[0].set_title(title1)

    axes[1].imshow(image2[frame], cmap="gray")
    axes[1].set_title(title2)

    for ax in axes:
        ax.axis('off')
    plt.tight_layout()
    plt.show()

#####################################
def plot_histogram(data, frame, range_values=(0, 30000), bins_count=1000, x_label="Grey Levels", y_label="Intensity [log scale]"):
    """
    Display histogram of the given data frame.

    Parameters:
    - data : numpy.ndarray : The 3D data array.
    - frame : int : The frame index to display.
    - range_values : tuple : The range of values for the histogram.
    - bins_count : int : Number of bins for the histogram.
    - x_label : str : X-axis label for the histogram.
    - y_label : str : Y-axis label for the histogram.

    Returns:
    None
    """
    fig, ax = plt.subplots()
    ax.hist(data[frame].flatten(), range=range_values, bins=bins_count, log=True)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    plt.show()
    
def apply_threshold_percentile(data, percentile=98):
    """
    Apply a threshold to the data using a given percentile.

    Parameters:
    - data (numpy.ndarray): Input data.
    - percentile (int): Percentile value to use for thresholding.

    Returns:
    - thresholded_data (numpy.ndarray): Thresholded data.
    """
    threshold_value = np.percentile(data, percentile)
    return (data > threshold_value).astype(int)

#######################################
def visualize_data_slice(data, slice_idx=40, cmap="gray"):
    """
    Display a 2D slice from the data.

    Parameters:
    - data (numpy.ndarray): Input data.
    - slice_idx (int): Index of the slice to display.
    - cmap (str): Colormap to use for display.
    """
    plt.imshow(data[slice_idx], cmap=cmap)
    plt.show()

#########################################
def save_data_as_images(data, output_folder, file_prefix="frame", file_format="png", dpi=300):
    """
    Save each slice of the data as an image.

    Parameters:
    - data (numpy.ndarray): 3D data array.
    - output_folder (str): Directory to save the images.
    - file_prefix (str): Prefix for image filenames.
    - file_format (str): Image file format.
    - dpi (int): Dots per inch for saved images.
    """
    os.makedirs(output_folder, exist_ok=True)

    for i in range(data.shape[0]):
        output_filename = os.path.join(output_folder, f'{file_prefix}_{i:04d}.{file_format}')
        plt.imsave(output_filename, data[i], dpi=dpi)

import os
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

def apply_closing(data, config):
    """
    Apply morphological closing operation on the data.

    Parameters:
    - data : numpy.ndarray : The 3D binary data array.
    - config : dict : Configuration dictionary containing 'SE', 'SE_t', and 'frame'.

    Returns:
    numpy.ndarray : The data after the closing operation.
    """
    se = np.ones((config['SE_t'], config['SE'], config['SE']))
    return ndimage.binary_opening(data, se).astype(data.dtype)

def display_image(data, frame, cmap="gray"):
    """
    Display the image from a specified frame.

    Parameters:
    - data : numpy.ndarray : The 3D data array.
    - frame : int : The frame index to display.
    - cmap : str : Colormap for the image display.

    Returns:
    None
    """
    plt.imshow(data[frame], cmap=cmap)
    plt.show()

def save_images_to_folder(data, folder_name):
    """
    Save each frame of the data as a PNG image in the specified folder.

    Parameters:
    - data : numpy.ndarray : The 3D data array.
    - folder_name : str : The name of the folder where the images will be saved.

    Returns:
    None
    """
    os.makedirs(folder_name, exist_ok=True)
    for i in range(data.shape[0]):
        output_filename = os.path.join(folder_name, f'frame_{i:04d}.png')
        plt.imsave(output_filename, data[i], dpi=300)
        
import napari
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from scipy.ndimage import median_filter
from skimage.io import imread
import matplotlib.pyplot as plt

# Initialize variables for cropping parameters
crop_x_start = 0
crop_x_end = None
crop_y_start = 0
crop_y_end = None
crop_t_start = 0
crop_t_end = None

# open Napari viewer
def open_napari_viewer(tiff_data):
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(tiff_data, name='TIFF Data')

# update the displayed image based on cropping parameters
def update_displayed_image():
    global crop_x_start, crop_x_end, crop_y_start, crop_y_end, crop_t_start, crop_t_end
    
    # Crop the data
    cropped_data = subset_astrocyte3d[
        crop_t_start:crop_t_end, crop_y_start:crop_y_end, crop_x_start:crop_x_end
    ]

    plt.imshow(cropped_data[frame_to_display], cmap="gray")
    plt.show()

# Create a function to open dialog for cropping parameters
def open_crop_dialog():
    global crop_x_start, crop_x_end, crop_y_start, crop_y_end, crop_t_start, crop_t_end

    # Create a separate dialog window for cropping
    crop_dialog = tk.Toplevel(root)
    crop_dialog.title("Crop Parameters")

    # Create labels an fields for cropping parameters
    x_start_label = tk.Label(crop_dialog, text="X Start:")
    x_start_entry = tk.Entry(crop_dialog)
    x_start_entry.insert(0, str(crop_x_start))
    x_start_label.grid(row=0, column=0)
    x_start_entry.grid(row=0, column=1)

    x_end_label = tk.Label(crop_dialog, text="X End:")
    x_end_entry = tk.Entry(crop_dialog)
    x_end_entry.insert(0, str(crop_x_end))
    x_end_label.grid(row=1, column=0)
    x_end_entry.grid(row=1, column=1)

    y_start_label = tk.Label(crop_dialog, text="Y Start:")
    y_start_entry = tk.Entry(crop_dialog)
    y_start_entry.insert(0, str(crop_y_start))
    y_start_label.grid(row=2, column=0)
    y_start_entry.grid(row=2, column=1)

    y_end_label = tk.Label(crop_dialog, text="Y End:")
    y_end_entry = tk.Entry(crop_dialog)
    y_end_entry.insert(0, str(crop_y_end))
    y_end_label.grid(row=3, column=0)
    y_end_entry.grid(row=3, column=1)

    t_start_label = tk.Label(crop_dialog, text="T Start:")
    t_start_entry = tk.Entry(crop_dialog)
    t_start_entry.insert(0, str(crop_t_start))
    t_start_label.grid(row=4, column=0)
    t_start_entry.grid(row=4, column=1)

    t_end_label = tk.Label(crop_dialog, text="T End:")
    t_end_entry = tk.Entry(crop_dialog)
    t_end_entry.insert(0, str(crop_t_end))
    t_end_label.grid(row=5, column=0)
    t_end_entry.grid(row=5, column=1)

    # Create a button to apply 
    apply_button = ttk.Button(
        crop_dialog, text="Apply", command=apply_crop_parameters
    )
    apply_button.grid(row=6, column=0, columnspan=2)

# Create a function to applyupdate 
def apply_crop_parameters():
    global crop_x_start, crop_x_end, crop_y_start, crop_y_end, crop_t_start, crop_t_end

    try:
        crop_x_start = int(x_start_entry.get())
        crop_x_end = int(x_end_entry.get())
        crop_y_start = int(y_start_entry.get())
        crop_y_end = int(y_end_entry.get())
        crop_t_start = int(t_start_entry.get())
        crop_t_end = int(t_end_entry.get())

        if any(val < 0 for val in [crop_x_start, crop_x_end, crop_y_start, crop_y_end, crop_t_start, crop_t_end]):
            raise ValueError("Crop values cannot be negative.")
        if crop_x_start >= crop_x_end or crop_y_start >= crop_y_end or crop_t_start >= crop_t_end:
            raise ValueError("Start values must be less than end values.")

        update_displayed_image()
    except ValueError as ve:
        messagebox.showwarning("Invalid Input", str(ve))
    except Exception as e:
        messagebox.showerror("Error", f"Failed to apply crop parameters: {e}")
        
import napari
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
from scipy.ndimage import median_filter
import numpy as np
import matplotlib.pyplot as plt
import tkinter.ttk as ttk  # Import ttk


crop_x_start = 0
crop_x_end = None
crop_y_start = 0
crop_y_end = None
crop_t_start = 0
crop_t_end = None

# Initialize global variables for median filtering and background subtraction
apply_median_filter = False
median_filter_frames = 3  # Default value
background_subtraction = False
background_subtraction_neighbors = 3  # Default value

# Create variables for entry fields
x_start_entry = None
x_end_entry = None
y_start_entry = None
y_end_entry = None
t_start_entry = None
t_end_entry = None
median_filter_checkbox = None
background_subtraction_checkbox = None

# function to apply median filtering and background subtraction
def apply_median_and_background():
    global apply_median_filter, apply_background_subtraction, median_filter_frames, background_subtraction_neighbors
    global crop_x_start, crop_x_end, crop_y_start, crop_y_end, crop_t_start, crop_t_end

    if apply_median_filter:
        median_filter_kernel = np.ones((median_filter_frames, 3, 3))
        median_result = apply_median_filtering(astrocyte3d, median_filter_kernel)

    if apply_background_subtraction:
        # Calculate background_image and background_subtracted_data
        background_image = calculate_background(astrocyte3d, background_subtraction_neighbors)
        background_subtracted_data = astrocyte3d - background_image

# open Napari viewer
def open_napari_viewer(tiff_data):
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(tiff_data, name='TIFF Data')

# open Napari viewer with cropped data
def open_napari_viewer_with_cropped_data(cropped_data):
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(cropped_data, name='Cropped Data')

# update the displayed image based on cropping parameters
def update_displayed_image():
    global crop_x_start, crop_x_end, crop_y_start, crop_y_end, crop_t_start, crop_t_end

# Create a function to apply the cropping parameters
def apply_crop_parameters():
    global crop_x_start, crop_x_end, crop_y_start, crop_y_end, crop_t_start, crop_t_end
    global x_start_entry, x_end_entry, y_start_entry, y_end_entry, t_start_entry, t_end_entry

    # Get values from entry fields and update cropping parameters
    crop_x_start = int(x_start_entry.get())
    crop_x_end = int(x_end_entry.get())
    crop_y_start = int(y_start_entry.get())
    crop_y_end = int(y_end_entry.get())
    crop_t_start = int(t_start_entry.get())
    crop_t_end = int(t_end_entry.get())

    # Update the displayed image
    update_displayed_image()

# Create a function to apply the median and background options
def apply_median_and_background_options():
    global apply_median_filter, apply_background_subtraction, median_filter_frames, background_subtraction_neighbors
    apply_median_filter = median_filter_checkbox.get()
    median_filter_frames = int(median_filter_frames_entry.get())
    apply_background_subtraction = background_subtraction_checkbox.get()
    background_subtraction_neighbors = int(background_subtraction_neighbors_entry.get())
    apply_median_and_background()
    
def apply_threshold(data, threshold_value):
    return data > threshold_value

# Function to be called when the button is clicked
def on_threshold_button_click():
    try:
        threshold_value = int(threshold_entry.get())
        global diffI_Md_MdBKG 
        global Threshold  

        Threshold = apply_threshold(diffI_Md_MdBKG, threshold_value)
        print("Threshold applied")

    except ValueError:
        print("Please enter a valid integer for the threshold")
        
def label_and_colorize():
    global Labelled, col, Threshold
    Con = 26  # connectivity
    Labelled = cc3d.connected_components(Threshold, connectivity=Con)
    col = colorize_segmentation(Labelled, ignore_label=0)

def fill_holes():
    global Closing, Threshold
    se = np.ones((5,2,2)) #SET STRUCTURE ELEMENT
    Closing = ndimage.binary_closing(Threshold, se).astype(Threshold.dtype)
    
def restrict_area():
    global  Labelled
    Area = 100 #SET MIN AREA/Volume SIZE
    Area_Restrict = remove_small_objects(Labelled, Area)
    
default_parameters = {
    'time_start': 0,
    'time_end': -1, 
    'y_start': 0,
    'y_end': -1,
    'x_start': 0,
    'x_end': -1,
    'threshold': 7000,
    'median_filter_size': (5, 3, 3),
    'BKG_Neighbors' : 660,
    'Connectivity' : 26,
    'Structure Element' : (5, 2, 2),
    'Area Size' : 100,
}

def crop_dataset(data, crop_params):
    time_start, time_end = crop_params['time_start'], crop_params['time_end']
    y_start, y_end = crop_params['y_start'], crop_params['y_end']
    x_start, x_end = crop_params['x_start'], crop_params['x_end']
    
    if time_end == -1: time_end = data.shape[0]
    if y_end == -1: y_end = data.shape[1]
    if x_end == -1: x_end = data.shape[2]
    
    return data[time_start:time_end, y_start:y_end, x_start:x_end]

def load_dataset_and_set_defaults():
    global automatic_parameters, default_parameters

    # Load the dataset
    astrocyte3d = imread('/export/scratch/mkunfeld/DataSets/3D/ADRA2a219-SSCx-roi1-Sham7d-Zm69-LoIso-Baseline-3_Registered.tif')

    # Update default parameters with the dataset dimensions
    t_dim, y_dim, x_dim = astrocyte3d.shape
    default_parameters.update({
        'time_end': t_dim,
        'y_end': y_dim,
        'x_end': x_dim
    })

    # Copy default parameters to automatic parameters
    automatic_parameters = default_parameters.copy()

    return astrocyte3d


def load_dataset_and_update_defaults():
    file_path = filedialog.askopenfilename(title="Select Dataset", filetypes=[("TIFF Files", "*.tif"), ("All Files", "*.*")])
    if file_path:
        global automatic_parameters, default_parameters, astrocyte3d

        # Load the dataset
        astrocyte3d = imread(file_path)

        # Get the total number of frames in the dataset
        total_frames = astrocyte3d.shape[0]

        # Update default parameters with the dataset dimensions and frame count
        default_parameters.update({
            'time_end': total_frames,
            'y_end': astrocyte3d.shape[1],
            'x_end': astrocyte3d.shape[2],
            'BKG_Neighbors': total_frames  # Set BKG_Neighbors to the total frame count
        })

        # Copy default parameters to automatic parameters
        automatic_parameters = default_parameters.copy()
        return True
    return False

def open_napari_with_all_data():
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(astrocyte3d, name='Original Data')
        viewer.add_image(astromedian, name='Median Filtered')
        viewer.add_image(bg_image_median, name='Background')
        viewer.add_image(diffI_Md_MdBKG, name='Background Subtracted')
        viewer.add_image(Threshold, name='Thresholded')
        viewer.add_image(Closing, name='Holes Filled')
        viewer.add_image(Area_Restrict, name='Area Restricted')
        viewer.add_image(col, name='Final Labelled')
        
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
import napari
from tkinter import ttk  

astrocyte3d = None
automatic_parameters = {
    'time_start': 0,
    'time_end': -1,
    'y_start': 0,
    'y_end': -1,
    'x_start': 0,
    'x_end': -1,
    'threshold': 7000,
    'median_filter_size': (5, 3, 3),
    'BKG_Neighbors': 660,
    'Connectivity': 26,
    'Structure_Element': (5, 2, 2),
    'Area_Size': 100
}




def load_dataset():
    global astrocyte3d, automatic_parameters, input_file_path
    file_path = filedialog.askopenfilename(title="Select Dataset", filetypes=[("TIFF Files", "*.tif"), ("All Files", "*.*")])
    if file_path:
        input_file_path = file_path  # Store the input file path
        astrocyte3d = imread(file_path)
        messagebox.showinfo("Info", "Dataset loaded successfully.")

        # Update cropping parameters to the full dimensions of the loaded dataset
        t_dim, y_dim, x_dim = astrocyte3d.shape
        automatic_parameters.update({
            'time_end': t_dim,
            'y_end': y_dim,
            'x_end': x_dim
        })

def open_auto_params_dialog():
    global astrocyte3d, automatic_parameters

    dialog = tk.Toplevel(root)
    dialog.title("Automatic Processing Parameters")

    # Use a dictionary to hold the entry fields corresponding to each parameter
    param_entries = {}

    # Function to populate the dialog with entry fields
    def create_param_entry(param_name, default_value):
        frame = tk.Frame(dialog)
        label = tk.Label(frame, text=f"{param_name}:")
        entry = tk.Entry(frame)
        entry.insert(0, str(default_value))
        label.pack(side=tk.LEFT)
        entry.pack(side=tk.LEFT)
        frame.pack()
        param_entries[param_name] = entry

    # Create entry fields for all parameters including cropping dimensions
    for param, value in automatic_parameters.items():
        create_param_entry(param, value)

    # Function to reset parameters to default values
    def use_default_parameters():
        for param, value in default_parameters.items():
            param_entries[param].delete(0, tk.END)
            param_entries[param].insert(0, str(value))

    # Function to apply parameters
    def apply_parameters():
        global automatic_parameters
        for param in automatic_parameters:
            entry_value = param_entries[param].get()
            if param in ['Structure_Element', 'median_filter_size']:
                values = entry_value.strip('()').split(',')
                automatic_parameters[param] = tuple(map(int, values))
            elif param in ['time_start', 'time_end', 'y_start', 'y_end', 'x_start', 'x_end']:
                automatic_parameters[param] = int(entry_value) if entry_value != '-1' else -1
            else:
                automatic_parameters[param] = type(automatic_parameters[param])(entry_value)
        dialog.destroy()
        automatic_image_processing_steps()

    apply_button = tk.Button(dialog, text="Apply", command=apply_parameters)
    apply_button.pack()

    default_button = tk.Button(dialog, text="Use Default", command=use_default_parameters)
    default_button.pack()

    dialog.transient(root)  
    dialog.grab_set()     
    root.wait_window(dialog)  
    
    
def automatic_image_processing_steps():
    global astrocyte3d, automatic_parameters, progress, input_file_path
    if astrocyte3d is None:
        messagebox.showwarning("Warning", "No dataset loaded.")
        return

    # Create output directory in the same location as input file
    output_dir = os.path.join(os.path.dirname(input_file_path), 'output')
    png_dir = os.path.join(output_dir, 'PNGs')  
    os.makedirs(png_dir, exist_ok=True)

    def save_image(image, name, format="tif"):
        if format == "tif":
            filename = os.path.join(output_dir, f"{name}.{format}")
            imsave(filename, image)
        elif format == "png":
            for i, frame in enumerate(image):
                filename = os.path.join(png_dir, f"{name}_frame_{i:03d}.png")
                skimage.io.imsave(filename, frame)

    # Crop dataset
    astrocyte3d = crop_dataset(astrocyte3d, automatic_parameters)
    
    progress['value'] = 20  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Apply Median Filtering
    footprint = np.ones(automatic_parameters['median_filter_size'])
    astromedian = median_filter(astrocyte3d, footprint=footprint)
    save_image(astromedian, "Median_Filtered", format="tif")
    
    progress['value'] = 30  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Calculate the Background
    Nframes = automatic_parameters['BKG_Neighbors']
    frames = astromedian.shape[0]
    bg_image_median = np.zeros(astromedian.shape, 'uint16')
    for i in range(frames):
        if i - Nframes < 0:
            bg_image_median[i,:,:] = np.min(astromedian[0:(i+Nframes+1),:,:], axis=0)
        elif i + Nframes + 1 > frames:
            bg_image_median[i,:,:] = np.min(astromedian[(i-Nframes):,:,:], axis=0)
        else:
            bg_image_median[i,:,:] = np.min(astromedian[(i-Nframes):(i+Nframes+1),:,:], axis=0)

    # Background Subtraction
    diffI_Md_MdBKG = astromedian - bg_image_median
    save_image(diffI_Md_MdBKG, "Background Substracted", format="tif")
    save_image(bg_image_median, "Background", format="tif")
    
    progress['value'] = 40  # Update progress after a step
    root.update_idletasks()  # Refresh  GUI to reflect the change

    # Apply Threshold
    t = automatic_parameters['threshold']
    Threshold = (diffI_Md_MdBKG > t)
    save_image(Threshold, "Thresholded", format="tif")
    
    progress['value'] = 50  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Apply Connected Components Labeling
    Con = automatic_parameters['Connectivity']
    Labelled = cc3d.connected_components(Threshold, connectivity=Con)
    col = colorize_segmentation(Labelled, ignore_label=0)
    
    progress['value'] = 60  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Fill Holes
    se = np.ones(automatic_parameters['Structure_Element'])
    Closing = ndimage.binary_closing(Threshold, se).astype(Threshold.dtype)
    save_image(Closing, "Holes Filled", format="png")
    
    progress['value'] = 70  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Restrict Area Size
    Area = automatic_parameters['Area_Size']
    Area_Restrict = remove_small_objects(Labelled, Area)
    
    progress['value'] = 80  # Update progress after a step
    root.update_idletasks()  # Refresh GUI to reflect the change

    # Labeling
    Labelled = cc3d.connected_components(Area_Restrict, connectivity=Con)
    col = colorize_segmentation(Labelled, ignore_label=0)
    save_image(col, "Area Restricted", format="png")
    
    progress['value'] = 100  # Set to 100% when processing is complete
    root.update_idletasks()

    # Open results in Napari
    with napari.gui_qt():
        viewer = napari.Viewer()
        viewer.add_image(astrocyte3d, name='Original Data')
        viewer.add_image(astromedian, name='Median Filtered')
        viewer.add_image(bg_image_median, name='Background')
        viewer.add_image(diffI_Md_MdBKG, name='Background Subtracted')
        viewer.add_image(Threshold, name='Thresholded')
        viewer.add_image(Closing, name='Holes Filled')
        viewer.add_image(col, name='Final Labelled')
        
    progress['value'] = 0  # Reset the progress bar after processing


def create_main_window():
    global root, progress  
    root = tk.Tk()
    root.title("Automatic Image Processing")

    load_button = tk.Button(root, text="Load Dataset", command=load_dataset)
    load_button.pack()

    params_button = tk.Button(root, text="Set Parameters & Process", command=open_auto_params_dialog)
    params_button.pack()

    #process_button = tk.Button(root, text="Start Processing", command=automatic_image_processing_steps)
    #process_button.pack()
    
    # Create progress bar
    progress = ttk.Progressbar(root, orient=tk.HORIZONTAL, length=200, mode='determinate')
    progress.pack()

    return root

if __name__ == '__main__':
    root = create_main_window()
    root.mainloop()


C:\Users\User\AppData\Local\Temp\ipykernel_16020\2025275885.py:1113: UserWarning: C:/Users/User/Documents/Master Thesis/Project/Results/PART1 - BASELINE/IN WORK/FIG1 - Noise Analysis of my DataSet [FINISHED]/Original Files\output\Thresholded.tif is a boolean image: setting True to 255 and False to 0. To silence this warning, please convert the image using img_as_ubyte.
  imsave(filename, image)
C:\Users\User\AppData\Local\Temp\ipykernel_16020\2025275885.py:1113: UserWarning: C:/Users/User/Documents/Master Thesis/Project/Results/PART1 - BASELINE/IN WORK/FIG1 - Noise Analysis of my DataSet [FINISHED]/Original Files\output\Thresholded.tif is a low contrast image
  imsave(filename, image)
C:\Users\User\AppData\Local\Temp\ipykernel_16020\2025275885.py:1117: UserWarning: C:/Users/User/Documents/Master Thesis/Project/Results/PART1 - BASELINE/IN WORK/FIG1 - Noise Analysis of my DataSet [FINISHED]/Original Files\output\PNGs\Holes Filled_frame_000.png is a boolean image: setting True to 255 and 

C:\Users\User\AppData\Local\Temp\ipykernel_16020\2025275885.py:1117: UserWarning: C:/Users/User/Documents/Master Thesis/Project/Results/PART1 - BASELINE/IN WORK/FIG1 - Noise Analysis of my DataSet [FINISHED]/Original Files\output\PNGs\Area Restricted_frame_001.png is a low contrast image
  skimage.io.imsave(filename, frame)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\User\AppData\Local\Temp\ipykernel_16020\2025275885.py:1117: UserWarning: C:/Users/User/Documents/Master Thesis/Project/Results/PART1 - BASELINE/IN WORK/FIG1 - Noise Analysis of my DataSet [FINISHED]/Original Files\output\PNGs\Area Restricted_frame_002.png is a low contrast image
  skimage.io.imsave(filename, frame)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\User\AppData\Local\Temp\ipykernel_16020\2025275885.py:1117: UserWarning: C:/Users/User/Documents/Mast

Assistant skips harvesting pyclesperanto as it's not installed.
